In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q -U tensorflow-addons
# install EfficientNet
!pip install -q efficientnet

In [ ]:
import math, re, os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras import regularizers   

print("Tensorflow version " + tf.__version__)

In [ ]:
# Detect hardware TPU and return appropriate distribution strategy
# TFRecord is a binary file that contains sequences of byte-strings.
try:
    # TPU detection
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default distribution strategy in Tensorflow
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

# We'll use the distribution strategy when we create our neural network model. 
# Then, TensorFlow will distribute the training among the eight TPU cores 
# by creating eight different replicas of the model, one for each core.

In [ ]:
from kaggle_datasets import KaggleDatasets  # import kaggle data files

# store datasets into google cloud service
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH) 
print('Entries in the bucket:')
!gsutil ls $GCS_DS_PATH # list items in the bucket 



In [ ]:
# parameters set for tfrecords-jpeg-512x512 TFRecord files in the bucket
IMAGE_SIZE = [512, 512]
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-512x512'
AUTO = tf.data.experimental.AUTOTUNE
HEIGHT             = IMAGE_SIZE[0]
WIDTH              = IMAGE_SIZE[1]
EPOCHS             = 12
# Define the batch size. This will be 16 with TPU off and 128 (=16*8) with TPU on
BATCH_SIZE         = 16 * strategy.num_replicas_in_sync

# current competition data: "tpu-getting-started" 
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') 

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAIN_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VAL_IMAGES   = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES  = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH  = NUM_TRAIN_IMAGES // BATCH_SIZE
AUTO             = tf.data.experimental.AUTOTUNE

print('No. of training images:')
print (NUM_TRAIN_IMAGES)
print('No. of validation images:')
print (NUM_VAL_IMAGES)
print('No. of testing images:')
print (NUM_TEST_IMAGES)


In [ ]:
CLASSES = [
    'pink primrose',        'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',      'wild geranium',         # 00-04
    'tiger lily',           'moon orchid',               'bird of paradise', 'monkshood',      'globe thistle',         # 05-09
    'snapdragon',           "colt's foot",               'king protea',      'spear thistle',  'yellow iris',           # 10-14
    'globe-flower',         'purple coneflower',         'peruvian lily',    'balloon flower', 'giant white arum lily', # 15-19
    'fire lily',            'pincushion flower',         'fritillary',       'red ginger',     'grape hyacinth',        # 20-24
    'corn poppy',           'prince of wales feathers',  'stemless gentian', 'artichoke',      'sweet william',         # 25-29
    'carnation',            'garden phlox',              'love in the mist', 'cosmos',         'alpine sea holly',      # 30-34
    'ruby-lipped cattleya', 'cape flower',               'great masterwort', 'siam tulip',     'lenten rose',           # 35-39
    'barberton daisy',      'daffodil',                  'sword lily',       'poinsettia',     'bolero deep blue',      # 40-44
    'wallflower',           'marigold',                  'buttercup',        'daisy',          'common dandelion',      # 45-49
    'petunia',              'wild pansy',                'primula',          'sunflower',      'lilac hibiscus',        # 50-54
    'bishop of llandaff',   'gaura',                     'geranium',         'orange dahlia',  'pink-yellow dahlia',    # 55-59
    'cautleya spicata',     'japanese anemone',          'black-eyed susan', 'silverbush',     'californian poppy',     # 60-64
    'osteospermum',         'spring crocus',             'iris',             'windflower',     'tree poppy',            # 65-69
    'gazania',              'azalea',                    'water lily',       'rose',           'thorn apple',           # 70-74
    'morning glory',        'passion flower',            'lotus',            'toad lily',      'anthurium',             # 75-79
    'frangipani',           'clematis',                  'hibiscus',         'columbine',      'desert-rose',           # 80-84
    'tree mallow',          'magnolia',                  'cyclamen ',        'watercress',     'canna lily',            # 85-89
    'hippeastrum ',         'bee balm',                  'pink quill',       'foxglove',       'bougainvillea',         # 90-94
    'camellia',             'mallow',                    'mexican petunia',  'bromelia',       'blanket flower',        # 95-99
    'trumpet creeper',      'blackberry lily',           'common tulip',     'wild rose'                                #100-103
]    

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image


def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs
def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
  # automatically interleaves reads from multiple file
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 
    
    # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order) 
    
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    dataset = dataset.map(read_labeled_tfrecord if labeled 
                          else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset


In [ ]:
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO)
    # statement in the next function (below), this happens essentially
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_brightness(image, max_delta=0.1) 
    image = tf.image.random_saturation(image, lower=0.7, upper=1.3)

    return image, label  

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES,labeled=True, ordered= ordered)
    dataset = dataset.cache()
    #dataset = dataset.shuffle(buffer_size=1920)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_test_dataset(ordered=False):  # order matters to submit predictions to Kaggle
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

print("Training:", ds_train)
print ("Validation:", ds_valid)
print("Test:", ds_test)

In [ ]:
np.set_printoptions(threshold=15, linewidth=80)

print("Training data shapes:")
for image, label in ds_train.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())
print("Test data shapes:")
for image, idnum in ds_test.take(3):
    print(image.numpy().shape, idnum.numpy().shape)  # idnum here is the unique identifier given to the image that we'll use later when we make our submission as a csv file.
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

In [ ]:
from matplotlib import pyplot as plt

def batch_to_numpy_images_and_labels(databatch):
    images, labels = databatch
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()  
    class_labels = []
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        class_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    else:
        for num in enumerate(numpy_labels):
            class_labels.append(CLASSES[num[1]])
    return numpy_images, class_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)

def show_images(databatch, row=6, col=8):  # row, col of subplots
    FIGSIZE = (col*3, row*3)  # 3X3 inch per image
    plt.figure(figsize=FIGSIZE)
    images, num_labl = batch_to_numpy_images_and_labels(databatch)
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.title(num_labl[j])
        plt.imshow(images[j])
    plt.show()

    

    
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square
    # or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()


    
    
    
    
    

def show_data_aug(image):
    ROW=len(images)
    COL=7  # 1 no-aug plus 6 aug images
    plt.figure(figsize=(COL*2,ROW*2))
    i=0
    for image in images:
        plt.subplot(ROW,COL,i*COL+1)
        plt.title('Flip L/R')
        plt.axis('off')  
        # augmented with random flip
        plt.imshow(tf.image.random_flip_left_right(image))       

        plt.subplot(ROW,COL,i*COL+2)
        plt.title('Resize & Crop')
        plt.axis('off')    
        # Pad the image with a black, 90-pixel border
        image1 = tf.image.resize_with_crop_or_pad(
            image, HEIGHT + 180, WIDTH + 180
        )
        # Randomly crop to original size from the padded image
        image1 = tf.image.random_crop(image1, size=[*IMAGE_SIZE,3])
        plt.imshow(image1)

        plt.subplot(ROW,COL,i*COL+3)
        plt.title('Contrast')
        plt.axis('off')
        # augmented with contrast
        plt.imshow(tf.image.random_contrast(image, 0.8, 1.2))  

        plt.subplot(ROW,COL,i*COL+4)
        plt.title('Brightness')
        plt.axis('off')
        # augmented with brightness
        plt.imshow(tf.image.random_brightness(image, 0.1))       

        plt.subplot(ROW,COL,i*COL+5)
        plt.title('No Augmentation')
        plt.axis('off')
        plt.imshow(image)

        plt.subplot(ROW,COL,i*COL+6)
        plt.title('Saturation')
        plt.axis('off')
        # augmented with saturation
        plt.imshow(tf.image.random_saturation(image, 0.7, 1.3))  

        plt.subplot(ROW,COL,i*COL+7)
        plt.title('Blur')
        plt.axis('off')        
        # ouput a value from a normal distribtion 
        rdm_filter2d = tf.random.normal([1], mean=0, stddev=1, dtype=tf.float32)              
        if rdm_filter2d > 2.0:  # 2 stddev above mean  
            # blur 2.5% of the images
            # using tfa.image mean filter
            plt.imshow(tfa.image.mean_filter2d(image, filter_shape = 3,padding='constant'))  
        else:
            plt.imshow(image)

        i+=1
        
    plt.show()

In [ ]:
get_training_dataset = (next(iter(ds_train.unbatch().batch(16)))) # get a batch for 
images, _ = batch_to_numpy_images_and_labels(get_training_dataset)

print('Training Dataset')
print('Image Augmentation with tf.image and tfa.image')
#one_batch = next(ds_iter)
#display_batch_of_images(one_batch)
show_data_aug(images)

In [ ]:
# Visualize effects before implementing in
# tf.keras.layers.experimental.preprocessing.Random___()
print('Training Dataset')
print('Image Augmentation with tf.keras.preprocesing.image.random')
ROW=len(images)
COL=4  # 1 no-augmentation plus 3 augmented images
plt.figure(figsize=(COL*3.5,ROW*3))
i=0
for image in images:
    plt.subplot(ROW,COL,i*4+1)
    plt.title('No Augmentation')
    plt.axis('off')
    plt.imshow(image)
    
    plt.subplot(ROW,COL,i*4+2)
    plt.title('Shift')
    plt.axis('off')
    # random shift on one numpy image tensor 
    # compared to tf.keras.layers.experimental.preprocessing.RandomTranslation(...)
    image2 = tf.keras.preprocessing.image.random_shift(
        image, wrg=0.15, hrg=0.15, row_axis=1, col_axis=2, channel_axis=2,
        fill_mode='constant'
    )    
    plt.imshow(image2) 

    plt.subplot(ROW,COL,i*4+3)
    plt.title('45-degree Rotation')
    plt.axis('off')
    # random rotation on one numpy image tensor
    # compared to tf.keras.layers.experimental.preprocessing.RandomRotation(...)
    image3 = tf.keras.preprocessing.image.random_rotation(
        image, rg=45, row_axis=1, col_axis=2, channel_axis=2, fill_mode='constant'
    )
    plt.imshow(image3)

    plt.subplot(ROW,COL,i*4+4)
    plt.title('Zoom')
    plt.axis('off')
    # random zoom on one numpy image tensor
    # comapred to tf.keras.layers.experimental.preprocessing.RandomZoom(...)
    image4 = tf.keras.preprocessing.image.random_zoom(
        image, (.75, 1.0), row_axis=1, col_axis=2, channel_axis=2, fill_mode='constant'
    )
    plt.imshow(image4)
    i+=1
plt.show()

In [ ]:
explore_img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=45, width_shift_range=0.15, height_shift_range=0.15,
    brightness_range=None, zoom_range=[0.75, 1.0], fill_mode='constant', 
    horizontal_flip=True, preprocessing_function=None
)

print('Training Dataset')
print('Image Augmentation with random transform method with ImageDataGenerator')
i = 0
ROW=8  # no. of rows
COL=4  # no . of cols
plt.figure(figsize=(COL*3.5,ROW*3))
for im in images:
    plt.subplot(ROW,COL,i*2+1)
    plt.title('No Augmentation')
    plt.axis('off')
    plt.imshow(im)
    plt.subplot(ROW,COL,i*2+2)
    plt.title('Transform With Image Gen.')
    plt.axis('off')
    plt.imshow(explore_img_gen.random_transform(im))
    i+=1
plt.show()

In [ ]:
R = 7     # rows of subplots/images
C = 6     # cols of subplots/images
B = R*C   # number of images in a batch

print('Training Images WITH Random Data Augmentation')
show_data_aug(next(iter(ds_train.unbatch().batch(B))))

In [ ]:
validation_dataset = get_validation_dataset(VALIDATION_FILENAMES)
# you may run these lines multiple times to view different samples from the image sets
print('Validation Images')
show_images(next(iter(validation_dataset.unbatch().batch(B))), row=R, col=C)

In [ ]:
# this needs TPU to run
# randomly shuffles the test for visualization
print('Test Images - Shuffled')
show_images(next(iter(ds_test.shuffle(buffer_size=NUM_TEST_IMAGES).unbatch().batch(B))), 
            row=R, col=C)

In [ ]:
# With pretrained model: DenseNet201
with strategy.scope():    
    pretrained_model = efn.EfficientNetB7(
        weights='imagenet', 
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrained_model.trainable = True # transfer learning
    model = tf.keras.Sequential([
        pretrained_model,                   
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES), kernel_regularizer=regularizers.L2(0.001), 
            activation='softmax')
    ])

# display model summary
model.summary()
    

In [ ]:
model.compile(
    optimizer='adam', #implemet adam algorithm
    loss = 'sparse_categorical_crossentropy', #specifies that crossentropy metric is computed between the labels and predictions
    metrics=['sparse_categorical_accuracy'] #metric is set to use sparse categorical accuracy, which calculates how often predictions matches the integer labels
)

In [ ]:
# Learning Rate Schedule Callback
# define a fine-tuned schedule for the Learning Rate Scheduler 
def exponential_lr(epoch,
                  start_lr=0.00001,min_lr=0.00001,max_lr=0.00005,
                  rampup_epochs = 5, sustain_epochs = 0,
                  exp_decay = 0.8):
    def lr(epoch, start_lr, min_lr,max_lr,rampup_epochs,sustain_epochs,
          exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr= ((max_lr-start_lr)/
                rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr 
        else:
            lr = ((max_lr - min_lr)* exp_decay ** (epoch-rampup_epochs-sustain_epochs)
                  + min_lr)
            
        return lr
    return lr(epoch,start_lr,min_lr,max_lr,rampup_epochs,sustain_epochs,exp_decay)

# set learning rate scheduler for callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(schedule=exponential_lr,verbose=True)

# learning rate chart
epoch_rng = [i for i in range(EPOCHS+15)] 
y = [exponential_lr(x) for x in epoch_rng]
plt.plot(epoch_rng,y)
plt.xlim(-1, EPOCHS+15)

print("Learning rate schedule: start = {:.3g}; peak = {:.3g}; end = {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
# EarlyStopping callback
# Stop training when a monitored metric has stopped improving
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# training stop safter 3 epochs if no improvement

In [ ]:
#we have already defined epochs in the start

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback, es_callback],
)

In [ ]:
# Create plots of loss and accuracy on the training and validation datasets

acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(history.history['loss'])+1)

plt.figure(figsize=(14, 14))
plt.subplot(2, 1, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')

plt.subplot(2, 1, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
#evaluate predictions with confusion matrix
def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

cmdataset = get_validation_dataset(ordered=True)
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch()

cm_correct_labels = next(iter(labels_ds.batch(NUM_VAL_IMAGES))).numpy()
cm_probabilities = model.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)

labels = range(len(CLASSES))
cmat = confusion_matrix(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
)
cmat = (cmat.T / cmat.sum(axis=1)).T # normalize

In [ ]:
score = f1_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
precision = precision_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
recall = recall_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
display_confusion_matrix(cmat, score, precision, recall)

In [ ]:
dataset = get_validation_dataset()
dataset = dataset.unbatch().batch(16)
batch = iter(dataset)

In [ ]:
test_ds = get_test_dataset(ordered=True)

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

In [ ]:
#Create file to submit to competitiomn
print('Generating submission.csv file...')

# Get image ids from test set and convert to unicode
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv